# Shap for YM

In [ ]:
import pandas as pd
import xgboost as xgb
import shap
# Initialize your Jupyter notebook with initjs(), otherwise you will get an error message.
shap.initjs()
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
import numpy as np

In [ ]:
# Define our search space for grid search
random_grid = {
    'objective' : ['reg:squarederror'],
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'max_depth': range(3, 10),
    'colsample_bytree': [i/10.0 for i in range(1, 3)],
    }

In [ ]:
#Used to fix the shap error
def myfun(self=None):
        return model_bytearray

In [ ]:
def non_zero_variables(df):
    cols = df.columns
    cols_non_0 = []
    for col in cols:
        if ((df[col].eq(0).sum(axis=0)) < 0.4 * (len(df[col]))): #if we have more than 60% zeros
            cols_non_0.append(col)
    return cols_non_0

### Train the model and get the 1-month-ahead predictions through the SHAP methodology:

In [ ]:
for country in ['YM']: 
     
    #Read all variables
    all_var = pd.read_csv('../../all_variables_and_GPI_monthly_all_countries/all_variables_%s.csv' 
                          %country, index_col = 0)

    
    #Delete all columns that have more than 60% of their values 0
    variables_non_0 = non_zero_variables(all_var) #Filter the variables that have many zeros
    df_country = all_var[variables_non_0]
    
    print(country, len(all_var))

    df_country = all_var
    
    df_country = df_country.loc[df_country.index >= 201503]
    
    train_set = 0.6
    
    #Set the target variable
    Y = df_country['GPI']

    #Set the independent variables
    X = df_country.loc[:, df_country.columns != 'GPI']

    #Set the training sets:
    Y_train = Y[:int(Y.shape[0]*train_set)]
    X_train = X[:int(X.shape[0]*train_set)]
    
    #Set the test sets
    X_test = X[int(X.shape[0]*train_set):]
    Y_test = Y[int(Y.shape[0]*train_set):]

    all_preds = []
    
    for i in range(0, len(X_test)):
        
        print(i)
    
        tscv = TimeSeriesSplit(n_splits=10).split(X_train)

        #Train the model

        xgb_reg = xgb.XGBRegressor() #model to tune

        xgb_reg_random = GridSearchCV(estimator = xgb_reg, param_grid = random_grid,
                                       cv = tscv,  n_jobs = -1)

        #Best model
        bmodel = xgb_reg_random.fit(X_train, Y_train)   

        #Pull the best estimated model from the gridsearch and send it to the TreeExplainer
        model = bmodel.best_estimator_

        #Fix a shap error
        mybooster = model.get_booster()
        model_bytearray = mybooster.save_raw()[4:]
        mybooster.save_raw = myfun
        #Finish Fix a shap error

        pred = model.predict(X_test.iloc[[i]])
        all_preds.append(pred[0])
        
        #Set the new training sets
        X_train = X_train.iloc[1:]
        X_train = X_train.append(X_test.iloc[i])
        Y_train = Y_train[1:]
        Y_train = Y_train.append(pd.Series(Y_test.iloc[i]))

In [ ]:
df_preds = pd.DataFrame(all_preds, columns = ['prediction1'])

In [ ]:
df_preds.to_csv('../../%s_shap_predictions.csv' %country)

Variables importance analysis through SHAP:

In [ ]:
for country in ['YM']: 
     
    #Read all variables
    all_var = pd.read_csv(path + 'all_variables_%s.csv' %country, index_col = 0)

    #Delete all columns that have more than 60% of their values 0
    variables_non_0 = non_zero_variables(all_var) #Filter the variables that have many zeros
    df_country = all_var[variables_non_0]
    
    print(country, len(all_var))

    df_country = all_var
    X_future = df_country.loc[df_country.index >= 201806]
    df_country = df_country.loc[df_country.index <= 201805]
    
    #Set the target variable
    Y = df_country['GPI']

    #Set the independent variables
    X = df_country.loc[:, df_country.columns != 'GPI']

    #Set the training sets:
    X_train = X.tail(36) #corresponds to the length of the dataset
    Y_train = Y.tail(36) #corresponds to the length of the dataset
    
    tscv = TimeSeriesSplit(n_splits=10).split(X_train)

    #Train the model

    xgb_reg = xgb.XGBRegressor() #model to tune

    xgb_reg_random = GridSearchCV(estimator = xgb_reg, param_grid = random_grid,
                                   cv = tscv,  n_jobs = -1)

    #Best model
    bmodel = xgb_reg_random.fit(X_train, Y_train)   
    
    #Pull the best estimated model from the gridsearch and send it to the TreeExplainer
    model = bmodel.best_estimator_
    
    #Fix a shap error
    mybooster = model.get_booster()
    model_bytearray = mybooster.save_raw()[4:]
    mybooster.save_raw = myfun
    #Finish Fix a shap error

In [ ]:
#Get the shap values
shap_values = shap.TreeExplainer(mybooster).shap_values(X_train)

Global variable importance plot:

In [ ]:
shap.summary_plot(shap_values, X_train, alpha=1, plot_type="bar", max_display=10, show=False)
                           
plt.title('Global variable importance Yemen XGBoost model ', loc='right', fontsize=25, fontweight='bold' )

plt.savefig('../../shap_summary_bar_%s.pdf' %country, bbox_inches='tight')

plt.show()

Individual Shap Value Plot:

In [ ]:
def shap_plot(j):
    explainerModel = shap.TreeExplainer(mybooster)
   
    plt.rc('font', **font)
    shap_values_Model = explainerModel.shap_values(X_future[X_future.columns[1:]])
    p = shap.force_plot(explainerModel.expected_value, shap_values_Model[j], X_future[X_future.columns[1:]].iloc[[j]].round(0).astype(object), show=False,matplotlib=True)
                                      
    p.set_figwidth(40)
    p.set_figheight(4)
    
    plt.tick_params(axis='x', labelsize = 12)
    plt.suptitle('Yemen XGBoost model\n Prediction for June 2018', fontsize=25, fontweight='bold', x=0.3, y=1.23)
    p.savefig('../../shap_plot_%s.pdf' %country, bbox_inches='tight')
    return(p)

In [ ]:
shap_plot(0)